In [1]:
import pandas as pd

Retrieving our Data
===

The first step to this process is getting all the data. We found a website named [SportsBookReviewsOnline.com](https://www.sportsbookreviewsonline.com/scoresoddsarchives/ncaafootball/ncaafootballoddsarchives.htm) that had the scores of NCAA football games for multiple years. The data downloaded into an .xlsx file (Microsoft Excel), where it was easily output into a .csv file. The data was imported into a Python `Panda's DataFrame`, which is shown below.

This data was lightly validated by comparing a couple games between this database and [CBS sports](https://www.cbssports.com/college-football/scoreboard/FBS/2019/regular/1/) website. All the games we looked at matched up, so the data seems trustworthy. That said, I am going to make some modifications to the data to clean it up.

In [2]:
df = pd.read_csv("ncaa_football_2019.csv")
df.head()

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,824,291,V,Florida,7,0,10,7,24,8.5,7,-310,7.5
1,824,292,H,MiamiFlorida,3,10,0,7,20,49,46,250,23
2,824,293,V,Arizona,0,21,14,3,38,12,10.5,-400,8
3,824,294,H,Hawaii,14,14,7,10,45,71,71,300,38
4,824,309801,V,Villanova,0,27,0,7,34,34.5,35,230,20


I want to look at any teams that only appear once. If a team only appears once, there is a high likelyhood that it is a typo. I can manually investigate and change the data to be more accurate

In [3]:
# Getting the data that only occurs once
df[df.groupby('Team').Team.transform('count') == 1]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
1077,1005,322,H,NorthernIllinois,14,3,0,3,20,6,4,-170,1.5
1127,1005,372,H,Buffalo,0,7,0,7,20,55,51.5,135,22.5
1196,1005,310265,V,NORTHALABAMA,2,0,13,19,34,3.5,2,-130,4
1227,1005,310296,H,NichollsState,6,14,7,7,34,3,3-115,-160,pk
1246,1005,310315,V,SavannahState,7,9,3,0,19,NL,44.5,475,21.5
2604,1123,125,V,TexasAM,0,3,3,7,13,46,43.5,400,20
2639,1123,160,H,AppalachianState,7,7,14,7,35,28,28.5,-8000,13
2767,1123,310474,H,ArkansasPineBluff,10,21,0,14,45,14,14.5,-600,7
2771,1123,310478,H,KennesawState,7,7,14,14,42,31.5,31.5,-13500,11.5
3014,1220,310705,N,UW-Whitewater,0,0,7,7,14,68.5,69.5,400,29.5


After some searching of the database and google work, here is what I found out. The data should be:

1. NoIllinois
2. BuffaloU
3. NorthAlabama
4. NichollsSt
5. This entry is actually okay. This team only had one game recorded in this database since they are a D3 school
6. TexasA&M
7. AppalachianSt
8. ArkPineBluff
9. KennesawSt

Now, for UM-Whitewater and NorthCentralIllinois, I can remove them from the data set. They only played one game in this set, and it was against each other. Since they don't have any common opponents with the rest of the dataset, they are junk. Normally dropping rows from a Pandas dataframe could cause some issues, however here it won't since I am removing a whole game, aka 2 rows. We end with SavannahState being the only team to play 1 game, as we want.

In [4]:
df.iat[1077, 3] = 'NoIllinois'
df.iat[1127, 3] = 'BuffaloU'
df.iat[1196, 3] = 'NorthAlabama'
df.iat[1227, 3] = 'NichollsSt'
df.iat[2604, 3] = 'TexasA&M'
df.iat[2639, 3] = 'AppalachianSt'
df.iat[2767, 3] = 'ArkPineBluff'
df.iat[2771, 3] = 'KennesawSt'

df.drop(3014, inplace=True)
df.drop(3015, inplace=True)
df[df.groupby('Team').Team.transform('count') == 1]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
1246,1005,310315,V,SavannahState,7,9,3,0,19,NL,44.5,475,21.5


Getting What's Useful
---

Now, to put the data in a readable format. I ran 3 list comprehensions to get the data in a new `DataFrame` that is easy to read. I needed to get half of the teams in `team1s`, their opponents in `team2s`, and the scores in terms of the first team into `scores`. 

In [5]:
# Gets the team for even numbered rows
team1s = [x['Team'] for i, x in df.iterrows() if i%2 == 0]

# Gets the team for odd numbered rows, the opponenets
team2s = [x['Team'] for i, x in df.iterrows() if i%2 == 1]

# Goes through the table by 2's. Subtracts team 2's scores from team 1's
scores = [df.iloc[x]['Final'] - df.iloc[x + 1]['Final'] for x in range(0,df.shape[0],2)]

Assembling Clean Data
---

Now I can assemble the scores into a simple `DataFrame`. That way I can easily manipulate the data in Python, or export it so the group can use it in their preferred software. 

In [6]:
# Our dataframe
massey_df = pd.DataFrame()

# Adding in our lists
massey_df['Team 1'] = team1s
massey_df['Team 2'] = team2s
massey_df['Score'] = scores

massey_df.head()

,Team 1,Team 2,Score
0,Florida,MiamiFlorida,4
1,Arizona,Hawaii,-7
2,Villanova,Colgate,20
3,YoungstownSt,Samford,23
4,UCLA,CincinnatiU,-10


In [7]:
# And export as a CSV as well
massey_df.to_csv('team_scores.csv', index=False)